#Lecture d'un cours en temps limité

**Prérequis de ce notebook:**
* Simplex-Algorithm-Linear-Programming
* Branch-and-Bound-Diving-Contest
* Dual-Simplex-Method

**Introduction:** La décomposition $LU$ d'une matrice carrée $A$ consiste en une matrice triangulaire inférieure $L$ et une matrice triangulaire supérieure $U$. Elle permet d'écrire le système $Ax=b$ (où l'inconnue est $x$) sous la forme $LUx=b$. En posant $y=Ux$, on a l'équation $Ly=b$ où la première ligne ne dépend que de la première inconnue $y_1$, car $L$ est triangulaire inférieure. Une fois $y_1$ trouvée (par une simple division), on obtient $y_2$ grâce à $y_1$ et grâce à la 2e équation (qui ne fait intervenir que $y_1$ et $y_2$). Et ainsi de suite. Cette étape, qui permet de trouver $y$, s'appelle la descente. Une fois $y$ trouvé, on n'a plus qu'à remonter dans l'équation $Ux=y$ pour trouver enfin $x$. La méthode de **descente-remontée** est bien plus efficace que d'inverser $A$ pour résoudre $Ax=b$. L'équation $Ax=b$ intervient souvent en programmation linéaire: c'est la forme standard des contraintes. A chaque itération de l'algorithme du Simplexe, il faut résoudre une équation de ce type. Au lieu d'inverser $A$ à chaque fois, on utilise la décomposition $LU$. C'est en partie ce que font les algos informatiques tels que **Gurobi** et **CBC**. Mais lorsqu'on résout un problème linéaire à la main, c'est encore plus simple : au lieu de faire des calculs matriciels, il suffit de pivoter par rapport à l'équation saturée. L'objectif de ce notebook est donc d'utiliser l'algorithme du Simplexe (et éventuellement la méthode Branch-and-Bound) en cherchant à faire le moins de calculs possible. On en profitera pour apprendre à lire les logs de Gurobi.  

**Objectifs de ce notebook:**
* Comprendre comment l'algorithme du Simplexe est utilisé **en pratique**, en cherchant à **minimiser le nombre de calculs nécessaires**. Plus particulièrement, on a vu dans Simplex-Algorithm-Linear-Programming qu'à chaque itération de l'algorithme du Simplexe, on effectue un calcul matriciel de ce type:
$
\begin{bmatrix}
x \\
y \\
c \\
d
\end{bmatrix}
={\begin{bmatrix}
2 & 3 & 0 & 0 \\
-1 & 1 & 0 & 0 \\
1 & -2 & 1 & 0 \\
1 & 1 & 0 & -1
\end{bmatrix}}^{-1}\left(
\begin{bmatrix}
4 \\
3 \\
5 \\
-3
\end{bmatrix}
-\begin{bmatrix}
1 & 0 \\
0 & 1 \\
0 & 0 \\
0 & 0
\end{bmatrix}
\begin{bmatrix}
a \\
b
\end{bmatrix}\right)
$
c'est-à-dire de la forme $x_b=B^{-1}(b-Ex_{e})$
* Comprendre que dans le cas où on applique l'algo à la main, faire des pivots de Gauss est bien plus rapide que de faire des calculs matriciels.
* Apprendre à utiliser Julia avec Gurobi pour résoudre informatiquement des problèmes de recherche opérationnelle. Apprendre à interpréter le log renvoyé par le solveur. Comprendre que Gurobi ne fonctionne ni par pivot de Gauss ni sur les matrices initiales, mais cherche à simplifier le problème au maximum par un **presolve** puis applique une décomposition LU.
* Se rappeler que lorsque le problème est un PLNE (PL à nombres entiers), on applique l'algorithme branch-and-bound qui consiste d'abord à résoudre par le Simplexe la relaxation continue puis à ramifier selon les variables non entières. Se rappeler que dans chaque ramification, il est plus intéressant d'appliquer l'algorithme DUAL du Simplexe car on ajoute une contrainte à un problème dont les coûts réduits sont déjà $\leq 0$ (ce qui en fait un problème dual réalisable mais pas forcément primal réalisable).

**Exemple utilisé:** On imagine qu'on a un cours avec 4 chapitres numérotés de $0$ à $3$. On n'a le temps de lire que $2$ chapitres. Lire le chapitre 0 donne 1 point à l'exam, lire le chapitre 1 donne 2 pts, le chapitre 2 donne 3 points et le chapitre 3 donne 4 points. Mais il y a un système de prérequis : pour lire le chapitre 2, il faut absolument avoir lu avant les chapitres 0 et 1. Pour lire le chapitre 3, il faut avoir lu le chapitre 1. Quelle est la meilleure stratégie pour gagner le plus de points ?

La réponse à cette question est évidente (chapitre 1 puis chapitre 3), mais ce problème va nous servir à vérifier que la méthode du Simplexe fonctionne bien. Puis nous allons complexifier un peu ce problème pour le résolution informatique.

**Plan:** \
I. Résolution du PLNE par algo du Simplexe + Branch-and-Bound, en cherchant à pivoter au lieu de faire des calculs matriciels \
II. Résolution d'un PLNE similaire (mais beaucoup plus complexe : 13 chapitres) par Julia et Gurobi (qui utilise un **presolve** puis Branch-and-Bound avec Algo Dual et décomposition LU)

# I. Résolution du PLNE par algo du Simplexe + Branch-and-Bound, en cherchant à pivoter au lieu de faire des calculs matriciels

On définit $x_0 \in \{0,1\}$ de sorte que si le chapitre 0 est lu, alors $x_0=1$. Sinon, $x_0=0$. \

On définit de même $x_1, x_2$ et $x_3$ pour les chapitres suivants. \

$x_i \in \{0,1\}$ se reformule en $x_i \leq 1$ car on sait que le problème est un PLNE (PL à valeurs entières).  

Il faut considérer les contraintes de prérequis:
* $x_2 \leq x_0$
* $x_2 \leq x_1$
* $x_3 \leq x_1$

Enfin, n'oublions pas la contrainte sur le nombre de chapitres à lire:
* $x_0+x_1+x_2+x_3\leq 2$

Toutes ces contraintes se reformulent en égalités en introduisant des variables d'écart (voir Simplex-Algorithm-Linear-Programming).

Les contraintes s'écrivent donc sous forme standard:

$$
A \mathbf{x} = \mathbf{b}
$$

où

$$
A = \begin{pmatrix}
1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
-1 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & -1 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & -1 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0
\end{pmatrix}
$$

et

$$
\mathbf{x} = \begin{pmatrix}
x_0 \\
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
x_7 \\
x_8 \\
x_9 \\
x_{10} \\
x_{11}
\end{pmatrix}
$$

On choisit comme base de départ $(x_0,x_1,x_4,x_5,x_6,x_9,x_{10},x_{11})$.

In [10]:
from sympy import Matrix, symbols, simplify
x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11 = symbols('x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11')
A=Matrix([
    [1,1,1,1,0,0,0,0,0,0,0,1],
    [-1,0,1,0,1,0,0,0,0,0,0,0],
    [0,-1,1,0,0,1,0,0,0,0,0,0],
    [0,-1,0,1,0,0,1,0,0,0,0,0],
    [1,0,0,0,0,0,0,1,0,0,0,0],
    [0,1,0,0,0,0,0,0,1,0,0,0],
    [0,0,1,0,0,0,0,0,0,1,0,0],
    [0,0,0,1,0,0,0,0,0,0,1,0]
])
# On choisit comme base de départ (x0,x1,x4,x5,x6,x9,x10,x11)
# Sous-matrices B et E (attention à l'indexation qui commence à 0 en python)
B=A[:,[0,1,4,5,6,9,10,11]]
E=A[:,[2,3,7,8]]
expression = B.inv() * (Matrix([[2,0,0,0,1,1,1,1]]).T - E * (Matrix([[x2,x3,x7,x8]]).T))
x_b = simplify(expression)
# x0 x1 x4 x5 x6 x9 x10 x11
x_b

Matrix([
[            1 - x7],
[            1 - x8],
[      -x2 - x7 + 1],
[      -x2 - x8 + 1],
[      -x3 - x8 + 1],
[            1 - x2],
[            1 - x3],
[-x2 - x3 + x7 + x8]])

Remarque : nous sommes dans un cas "dégénéré" car la variable de base $x_5$ vaut $0$ (alors qu'elle est en base).

In [11]:
z=Matrix([[1, 2, 0, 0, 0, 0, 0, 0]])*x_b+Matrix([[3,4,0,0]])*Matrix([[x2,x3,x7,x8]]).T
z

Matrix([[3*x2 + 4*x3 - x7 - 2*x8 + 3]])

$x_2$ et $x_3$ ont un coût réduit $>0$ donc la base $(x_0,x_1,x_4,x_5,x_6,x_9,x_{10},x_{11})$ n'est pas optimale.

On choisit de faire entrer $x_3$ en base car c'est la variable ayant le coût réduit le plus élevé.

Cela sort instantanément $x_{11}$ de la base, car $x_{11}$ est la variable d'écart dont la positivité ($\geq 0$) contraint le plus l'augmentation de $x_3$.

Bien sûr, on pourrait entièrement refaire les calculs pour la nouvelle base comme ci-dessous...

In [12]:
# Nouvelle base : (x0,x1,x3,x4,x5,x6,x9,x10)
# Sous-matrices B et E (attention à l'indexation qui commence à 0 en python)
B=A[:,[0,1,3,4,5,6,9,10]]
E=A[:,[2,7,8,11]]
expression = B.inv() * (Matrix([[2,0,0,0,1,1,1,1]]).T - E * (Matrix([[x2,x7,x8,x11]]).T))
x_b = simplify(expression)
# x0 x1 x3 x4 x5 x6 x9 x10
x_b

Matrix([
[                  1 - x7],
[                  1 - x8],
[     -x11 - x2 + x7 + x8],
[            -x2 - x7 + 1],
[            -x2 - x8 + 1],
[x11 + x2 - x7 - 2*x8 + 1],
[                  1 - x2],
[  x11 + x2 - x7 - x8 + 1]])

... mais il est plus astucieux de seulement faire pivoter $x_3$ et $x_{11}$ dans le système de contraintes exprimé précédemment. Cela évite de redéfinir une matrice $B$, de l'inverser, et de tout recalculer. Effectuons le pivot de Gauss: $x_{11}=-x_{2}-x_{3}+x_{7}+x_{8}$ implique $x_3=-x_{2}+x_{7}+x_{8}-x_{11}$ puis en injectant dans le reste du système suivant:

$$
\left\{
\begin{aligned}
    x_0 &= 1-x_7 \\
    x_1 &= 1-x_8 \\
    x_4 &= 1-x_2-x_7 \\
    x_5 &= 1-x_2-x_8 \\
    x_6 &= 1-x_3-x_8 \\
    x_9 &= 1-x_2 \\
    x_{10} &= 1-x_3 \\
    x_3 &=-x_{2}+x_{7}+x_{8}-x_{11}
\end{aligned}
\right.
$$

$x_3$ n'apparaît que dans $x_6$ et $x_{10}$ donc on fait le pivot pour ces variables:

$$
\left\{
\begin{aligned}
    x_0 &= 1-x_7 \\
    x_1 &= 1-x_8 \\
    x_3 &=-x_{2}+x_{7}+x_{8}-x_{11} \\
    x_4 &= 1-x_2-x_7 \\
    x_5 &= 1-x_2-x_8 \\
    x_6 &= 1+x_{2}-x_{7}-2x_{8}+x_{11} \\
    x_9 &= 1-x_2 \\
    x_{10} &= 1+x_{2}-x_{7}-x_{8}+x_{11}
\end{aligned}
\right.
$$

Sans surprise, on obtient exactement les mêmes expressions que par le calcul matriciel, mais en ayant fait beaucoup moins de calculs !

SymPy peut aussi faire cette petite transformation:

In [13]:
from sympy import symbols, Eq, solve
equations = [
    Eq(x0,1-x7),
    Eq(x1,1-x8),
    Eq(x4,1-x2-x7),
    Eq(x5,1-x2-x8),
    Eq(x6,1-x3-x8),
    Eq(x9,1-x2),
    Eq(x10,1-x3),
    Eq(x11,-x2-x3+x7+x8)
]

x3_expr = solve(equations[7], x3)[0]  # exprimer x3 en fonction du reste dans l'équation 7
equations = [eq.subs(x3, x3_expr) for eq in equations[:-1]]  # substituer dans toutes les autres équations (en fait on redéfinit le système "equations")
equations.append(Eq(x3, x3_expr)) # on ajoute l'équation-pivot au nouveau système
for eq in equations:
    print(eq)

Eq(x0, 1 - x7)
Eq(x1, 1 - x8)
Eq(x4, -x2 - x7 + 1)
Eq(x5, -x2 - x8 + 1)
Eq(x6, x11 + x2 - x7 - 2*x8 + 1)
Eq(x9, 1 - x2)
Eq(x10, x11 + x2 - x7 - x8 + 1)
Eq(x3, -x11 - x2 + x7 + x8)


A partir de maintenant, nous ferons toujours un pivot de ce genre pour toutes les itérations (sauf la première où nous continuerons à inverser la matrice B).

In [14]:
# Fonction objectif à la 2e itération
obj=equations[0].rhs+2*equations[1].rhs+3*x2+4*equations[7].rhs   # x0 + x1 + x2 + x3
obj

-4*x11 - x2 + 3*x7 + 2*x8 + 3

$x_7$ et $x_8$ ont un coût réduit $>0$ donc la base $(x_0,x_1,x_3,x_4,x_5,x_6,x_9,x_{10})$ n'est pas primal-réalisable. $x_7$ a le coût réduit le plus élevé donc entre en base. L'entrée de $x_7$ fait sortir $x_0$ de la base.

Faisons pivoter les équations.

In [15]:
x7_expr = solve(equations[0], x7)[0]
equations = [eq.subs(x7, x7_expr) for eq in equations[1:8]]
equations.append(Eq(x7, x7_expr))
for eq in equations:
    print(eq)

Eq(x1, 1 - x8)
Eq(x4, x0 - x2)
Eq(x5, -x2 - x8 + 1)
Eq(x6, x0 + x11 + x2 - 2*x8)
Eq(x9, 1 - x2)
Eq(x10, x0 + x11 + x2 - x8)
Eq(x3, -x0 - x11 - x2 + x8 + 1)
Eq(x7, 1 - x0)


In [16]:
# Fonction objectif à la 3e itération
obj=x0+2*equations[0].rhs+3*x2+4*equations[6].rhs   # x0 + x1 + x2 + x3
obj

-3*x0 - 4*x11 - x2 + 2*x8 + 6

$x_8$ a un coût réduit $>0$ donc on le fait entrer en base. On fait sortir $x_6$, qui correspond à l'inégalité la plus contraignante.

In [17]:
x8_expr = solve(equations[3], x8)[0]
equations = [eq.subs(x8, x8_expr) for i, eq in enumerate(equations) if i != 3]
equations.append(Eq(x8, x8_expr))
for eq in equations:
    print(eq)

Eq(x1, -x0/2 - x11/2 - x2/2 + x6/2 + 1)
Eq(x4, x0 - x2)
Eq(x5, -x0/2 - x11/2 - 3*x2/2 + x6/2 + 1)
Eq(x9, 1 - x2)
Eq(x10, x0/2 + x11/2 + x2/2 + x6/2)
Eq(x3, -x0/2 - x11/2 - x2/2 - x6/2 + 1)
Eq(x7, 1 - x0)
Eq(x8, x0/2 + x11/2 + x2/2 - x6/2)


In [18]:
# Fonction objectif à la 4e itération
obj=x0+2*equations[0].rhs+3*x2+4*equations[5].rhs   # x0 + x1 + x2 + x3
obj

-2*x0 - 3*x11 - x6 + 6

Tous les coûts réduits sont $\leq 0$ (alors qu'on est en maximisation) donc l'optimum de la relaxation continue est atteint.

De plus, l'optimum est : $x_1=x_3=x_5=x_7=x_9=1$ (et le reste à 0). Cet optimum est entier donc c'est aussi l'optimum pour le problème initial (problème PLNE).

**Remarque:** Aucune itération de Branch-and-Bound n'a été nécessaire car l'optimum de la relaxation continue est entier. C'est une chance, et ce n'est pas toujours le cas.

Lorsque ce n'est pas le cas, il faut ramifier selon une variable $x_i$ non entière, puis résoudre chacun des 2 sous-problèmes. Cela ajoute une contrainte au problème. Cela a de grandes chances de rendre la base optimale (de la relaxation) non réalisable (car elle ne vérifie pas la dernière inégalité).

**Point important:** Si c'est le cas, **ne pas repartir de zéro en recommençant tout l'algo du simplexe**. Partir de la dernière formulation du problème (celle optimale pour la relaxation continue) et ajouter la dernière contrainte. On a directement une base non **primal-réalisable** (car la dernière contrainte n'est pas vérifiée) mais **dual-réalisable** (car les coûts réduits sont tous $\leq 0$). Appliquer **l'algorithme dual du simplexe** (voir Dual-Simplex-Method) pour atteindre l'optimum en à peine quelques pivots.

Ici, nous n'avons pas à le faire car la relaxation continue a une solution entière.

Dans le fichier Simplex-Chapters.jl, je résouds ce problème en utilisant Julia et Gurobi. Bien sûr, on obtient le même résultat que ci-dessus.

# II. Résolution d'un PLNE similaire (mais beaucoup plus complexe : 13 chapitres) par Julia et Gurobi (qui utilise un **presolve** puis Branch-and-Bound avec Algo Dual et décomposition LU)

Pour finir, je propose de résoudre le même problème avec un livre un peu plus gros:

Liste de chapitres : 0,1,2,3,4,5,6,7,8,9,10,11,12 \

Tableau des prérequis:

|Chapitre|Prérequis|
|--------|---------|
|0|NONE|
|1|NONE|
|2|NONE|
|3|0;1|
|4|2;3|
|5|0;1|
|6|4;5|
|7|6|
|8|2;4|
|9|7|
|10|8|
|11|1;4|
|12|10;11|

On veut maximiser $\sum_{i=0}^{12}(i+1)x_{i}$ sous les contraintes: \
* $x_i \in \{0,1\}$ pour tout $i$ (soit le chapitre est lu, soit il n'est pas lu)
* $\sum_{i=0}^{12}x_i\leq7$ (on ne peut lire que 7 chapitres)
* $x_3 \leq x_0$
* $x_3 \leq x_1$
* $x_4 \leq x_2$
* $x_4 \leq x_3$
* $x_5 \leq x_0$
* $x_5 \leq x_1$
* $x_6 \leq x_4$
* $x_6 \leq x_5$
* $x_7 \leq x_6$
* $x_8 \leq x_2$
* $x_8 \leq x_4$
* $x_9 \leq x_7$
* $x_{10} \leq x_8$
* $x_{11} \leq x_1$
* $x_{11} \leq x_4$
* $x_{12} \leq x_{10}$
* $x_{12} \leq x_{11}$

On ne va pas s'amuser à détailler tous les calculs. Utilisons Julia et Gurobi.
* Commençons par voir si la relaxation continue est à solution entière (comme pour l'exemple précédent), car dans ce cas il n'y a pas vraiment de "branch and bound" à effectuer. Voir le code dans le fichier Simplex-Chapters-2.jl. Malheureusement, on obtient $x_i^{*}=\frac{7}{13}$ pour tout $i$. La solution n'est pas entière.
* Nous allons maintenant utiliser la technique de branch-and-bound. Dans le code de Julia, il suffit de rajouter la mention ```Int``` au moment de définir les variables. On ajoute aussi la commande ```set_optimizer_attribute(model, "OutputFlag", 1)``` pour que solveur nous explique toutes les étapes effectuées. Voir branch-and-bound-chapters.jl
* Gurobi n'utilise pas de pivot car le pivot est vraiment une méthode manuelle. A la place, Gurobi utilise une méthode matricielle mais réduit au maximum la charge de calculs en faisant une décomposition LU.

Voici la réponse du solveur: \

```
Found heuristic solution: objective 28.0000000
Presolve removed 28 rows and 8 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 9 nonzeros
Variable types: 0 continuous, 5 integer (5 binary)
Found heuristic solution: objective 36.0000000

Root relaxation: objective 3.700000e+01, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0        36.00000   36.00000  0.00%     -    0s

Explored 1 nodes (2 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 36 28

Optimal solution found (tolerance 1.00e-04)
Best objective 3.600000000000e+01, best bound 3.600000000000e+01, gap 0.0000%

User-callback calls 310, time in user-callback 0.00 sec
optimal value of x0: 1.0
optimal value of x1: 1.0
optimal value of x2: 1.0
optimal value of x3: 1.0
optimal value of x4: 1.0
optimal value of x5: 0.0
optimal value of x6: 0.0
optimal value of x7: 0.0
optimal value of x8: 1.0
optimal value of x9: 0.0
optimal value of x10: 0.0
optimal value of x11: 1.0
optimal value of x12: 0.0
objective value: 36.0
```

Etudions chaque ligne du solveur:
* ```Found heuristic solution: objective 28.0000000``` signifie simplement que Gurobi a utilisé une méthode heuristique pour trouver rapidement une solution réalisable. Cette ligne n'est pas très importante, et ce n'est pas grave de ne pas savoir ce qu'est une heuristique dans un premier temps.
* ```Presolve removed 28 rows and 8 columns``` signifie que **Gurobi a trouvé des contraintes redondantes** et les a supprimées. Plus précisément, Gurobi a supprimé $28$ contraintes et $8$ variables. Ces variables supprimées ne sont pas forcément les $x_i$ ci-dessus, mais peuvent faire partie des variables d'écart introduites automatiquement par le solveur pour transformer les inégalités en égalités. Par exemple, $x_{12} \leq x_{11}$ et $x_{11} \leq 1$, donc la contrainte $x_{12} \leq 1$, inutile, a été supprimée avec la variable d'écart correspondante.  
* ```Presolved: 3 rows, 5 columns, 9 nonzeros``` signifie qu'après prétraitement, il ne reste plus que $3$ contraintes, $5$ variables et la matrice des contraintes (qui contient $3 \times 5=15$ cases) a $9$ cases non nulles (ce qui indique le niveau de "sparsité" de la matrice des contraintes).
* ```Found heuristic solution: objective 36.0000000``` : le système indique avoir réutilisé une méthode heuristique après le presolve.
* ```Root relaxation: objective 3.700000e+01, 2 iterations, 0.00 seconds (0.00 work units)``` signifie que Gurobi a utilisé l'algorithme du simplexe sur la relaxation continue **du problème réduit (post presolve)** et a atteint l'optimum en 2 itérations. En général, il n'y a pas de raison que la valeur optimale de la relaxation du problème presolve corresponde à la valeur optimale de la relaxation du problème initial.
*
```Nodes    |    Current Node    |     Objective Bounds      |     Work``` \
``` Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time```

```0     0 infeasible    0        36.00000   36.00000  0.00%     -    0s``` se lit plus clairement:

| Nodes         || Current Node      ||| Objective Bounds     ||| Work          ||
|---------------||-------------------|||-----------------------|||---------------||
| Expl | Unexpl | Obj | Depth | IntInf | Incumbent | BestBd | Gap | It/Node | Time |
| 0     | 0      | infeasible | 0     |-| 36.00000 | 36.00000 | 0.00% | - | 0s |

* ```Expl``` est le nombre de noeuds explorés : $0$
* ```Unexpl``` est le nombre de noeuds non explorés : $0$
* ```Obj``` est la valeur de la fonction objectif (ici ```infeasible``` car la base n'est pas réalisable)
* ```depth``` désigne la profondeur actuelle de l'arbre de recherche
* ```IntInf``` désigne le nombre de variables entières mais non entières dans la solution de base (donc nécessitant une ramification) : ici $0$
* ```Incumbent``` est la **meilleure solution ENTIERE** trouvée jusqu'ici
* ```BestBd``` est la **meilleure borne** dans l'algo de branch-and-bound
* ```Gap``` : écart entre la meilleure solution connue (dans N) et la meilleure borne (dans R)
* ```Time``` temps écoulé **depuis la première itération** (c'est un temps cumulé)
* Ici, on constate qu'avec le presolve et la méthode heuristique, on avait déjà atteint la solution donc le Branch And Bound n'a pas vraiment servi.

* ```Solution count 2: 36 28``` Deux solutions réalisables ont été trouvées : 28 et 36
* ```Best objective 3.600000000000e+01, best bound 3.600000000000e+01, gap 0.0000%``` : la solution optimale est $36$
* ```Explored 1 nodes (2 simplex iterations) in 0.00 seconds (0.00 work units)``` : noeuds parcourus et temps total du processus
* La solution optimale est obtenue pour $x_0=x_1=x_2=x_3=x_4=x_8=x_{11}=1$ et le reste à 0.


**Bibliographie:** Pour l'introduction et la décomposition $LU$, voir la page de Wikipédia qui explique très bien cette décomposition